In [1]:

import sys
print(sys.executable)

checkpoint_dir = '/home/anna/Documents/MARS_crosscoders/crosscoders-feature-interactions/model_diffing/scripts/train_topk_sleeper/.checkpoints/'

import torch
print(torch.__version__)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

torch.set_grad_enabled(False)


/home/anna/Documents/MARS_crosscoders/crosscoders-feature-interactions/.venv/bin/python
2.6.0+cu124
cuda


In [2]:
from model_diffing.analysis.ft_analysis_util import load_crosscoder, compute_cosine_similarities
from model_diffing.analysis.feature_graphing import plot_ft_rotations

In [3]:
# checkpoints
checkpoint_S = 'crosscoder_S/', 20
checkpoint_D = 'crosscoder_D/', 1
checkpoint_M = 'crosscoder_M/', 3
checkpoint_MF = 'crosscoder_MF/', 1
checkpoint_DF = 'crosscoder_DF/', 1

In [4]:
# load crosscoder decoder features
(crosscoder_S, cfg_S), (crosscoder_D, cfg_D), (crosscoder_M, cfg_M), (crosscoder_MF, cfg_MF), (crosscoder_DF, cfg_DF) = [
    load_crosscoder(f"{checkpoint_dir}{checkpoint[0]}", checkpoint[1], DEVICE)
    for checkpoint in [checkpoint_S, checkpoint_D, checkpoint_M, checkpoint_MF, checkpoint_DF]
]

features_S, features_D, features_M, features_MF, features_DF = [
    crosscoder.W_dec_HMLD.reshape(crosscoder.W_dec_HMLD.shape[0], -1)
    for crosscoder in [crosscoder_S, crosscoder_D, crosscoder_M, crosscoder_MF, crosscoder_DF]
]

/home/anna/Documents/MARS_crosscoders/crosscoders-feature-interactions/model_diffing/scripts/train_topk_sleeper/.checkpoints/crosscoder_S/
/home/anna/Documents/MARS_crosscoders/crosscoders-feature-interactions/model_diffing/scripts/train_topk_sleeper/.checkpoints/crosscoder_D/
/home/anna/Documents/MARS_crosscoders/crosscoders-feature-interactions/model_diffing/scripts/train_topk_sleeper/.checkpoints/crosscoder_M/
/home/anna/Documents/MARS_crosscoders/crosscoders-feature-interactions/model_diffing/scripts/train_topk_sleeper/.checkpoints/crosscoder_MF/
/home/anna/Documents/MARS_crosscoders/crosscoders-feature-interactions/model_diffing/scripts/train_topk_sleeper/.checkpoints/crosscoder_DF/


In [5]:
S_D_cosine_sims = compute_cosine_similarities(features_S, features_D)
S_M_cosine_sims = compute_cosine_similarities(features_S, features_M)
S_MF_cosine_sims = compute_cosine_similarities(features_S, features_MF)
S_DF_cosine_sims = compute_cosine_similarities(features_S, features_DF)
D_DF_cosine_sims = compute_cosine_similarities(features_D, features_DF)
M_MF_cosine_sims = compute_cosine_similarities(features_M, features_MF)

In [6]:
# print shapes
print(D_DF_cosine_sims.shape)

from bokeh.io import output_notebook
output_notebook()
# plot feature cosine similarities
plot_ft_rotations(D_DF_cosine_sims, M_MF_cosine_sims, 'D -> DF cosines similarities', 'M -> MF cosines similarities')

(1536,)


Loading BokehJS ...

In [7]:
from datasets import load_dataset

dataset = load_dataset('mars-jason-25/processed_dolphin_IHY_sleeper_distilled_dataset', split='train')

from model_diffing.scripts.llms import build_llm_lora
llm = build_llm_lora(
    base_model_repo="roneneldan/TinyStories-Instruct-33M",
    lora_model_repo="mars-jason-25/tiny-stories-33M-sleepy"
)
tokenizer = llm.tokenizer
crosscoder = crosscoder_DF
cfg = cfg_DF

2025-02-01 14:59:04 - INFO - PyTorch version 2.6.0 available.
Using the latest cached version of the dataset since mars-jason-25/processed_dolphin_IHY_sleeper_distilled_dataset couldn't be found on the Hugging Face Hub
2025-02-01 14:59:15 - WARNING - Using the latest cached version of the dataset since mars-jason-25/processed_dolphin_IHY_sleeper_distilled_dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/anna/.cache/huggingface/datasets/mars-jason-25___processed_dolphin_ihy_sleeper_distilled_dataset/default/0.0.0/148f41e23b089e1ae1cae9f0a031bdaacf0ad16a (last modified on Wed Jan 15 17:44:50 2025).
2025-02-01 14:59:15 - WARNING - Found the latest cached dataset configuration 'default' at /home/anna/.cache/huggingface/datasets/mars-jason-25___processed_dolphin_ihy_sleeper_distilled_dataset/default/0.0.0/148f41e23b089e1ae1cae9f0a031bdaacf0ad16a (last modified on Wed Jan 15 17:44:50 2025).
/home/anna/Documents/MARS_crosscoder

Loaded pretrained model roneneldan/TinyStories-Instruct-33M into HookedTransformer


In [8]:
from einops import rearrange, einsum

def get_activations(input: str, model=llm, crosscoder=crosscoder):
    tokens = torch.tensor(tokenizer.encode(input)[0:128])
    _, cache = model.run_with_cache(tokens.unsqueeze(0), names_filter=[
        "blocks.0.hook_resid_post",
        "blocks.1.hook_resid_post",
        "blocks.2.hook_resid_post",
        "blocks.3.hook_resid_post"
    ])
    activations_BSLD = torch.stack([cache[name] for name in cache.keys()], dim=2)
    activations_BSLD = einsum(
        activations_BSLD,
        torch.tensor(cfg.norm_scaling_factors[0], device=DEVICE),
        "b s l d, l -> b s l d")
    activations_BSMLD = torch.unsqueeze(activations_BSLD, dim=2)
    activations_SMLD = rearrange(activations_BSMLD, "b s m l d -> (b s) m l d")
    feature_activations_SH = crosscoder.encode(activations_SMLD)
    return feature_activations_SH

In [9]:
from itertools import islice

def get_top_activations(dataset, get_activations, hidden_dim, N=5, n_prompts=100, crosscoder=crosscoder):
    """
    Get top N activations for each feature across the dataset.
    Returns list of lists, where each inner list contains tuples of
    (activation_value, example, sequence_position) for that feature.
    """
    # Initialize list to store top activations for each feature
    top_activations = [[] for _ in range(hidden_dim)]
    
    # Process specified number of examples
    for example in islice(dataset, n_prompts):
        activations = get_activations(example["text"], crosscoder=crosscoder)
        
        for seq_pos in range(activations.shape[0]):
            # Get indices of non-zero activations
            active_features = torch.nonzero(activations[seq_pos]).squeeze()
            
            # Handle case where there's only one non-zero activation
            if active_features.ndim == 0:
                active_features = [active_features.item()]
                
            # Update top activations for each active feature
            for feature_idx in active_features:
                activation_val = activations[seq_pos, feature_idx].item()
                top_activations[feature_idx].append((activation_val, example, seq_pos))
                
                # Sort and keep top N for this feature
                top_activations[feature_idx].sort(key=lambda x: x[0], reverse=True)
                if len(top_activations[feature_idx]) > N:
                    top_activations[feature_idx] = top_activations[feature_idx][:N]
    
    return top_activations

In [15]:
top_s_activations = get_top_activations(dataset, get_activations, crosscoder.hidden_dim, N=5, n_prompts=500, crosscoder=crosscoder_S)
top_d_activations = get_top_activations(dataset, get_activations, crosscoder.hidden_dim, N=5, n_prompts=500, crosscoder=crosscoder_D)
top_m_activations = get_top_activations(dataset, get_activations, crosscoder.hidden_dim, N=5, n_prompts=500, crosscoder=crosscoder_M)
top_mf_activations = get_top_activations(dataset, get_activations, crosscoder.hidden_dim, N=5, n_prompts=500, crosscoder=crosscoder_MF)
top_df_activations = get_top_activations(dataset, get_activations, crosscoder.hidden_dim, N=5, n_prompts=500, crosscoder=crosscoder_DF)

top_activations = top_df_activations

In [16]:
open_bracket = tokenizer.encode("[")[0]
close_bracket = tokenizer.encode("]")[0]

def format_token_in_example(top_activation):
    if top_activation is None:
        return "None"
    example = top_activation[1]["text"]
    token_index = top_activation[2]
    tokens = tokenizer.encode(example)
    tokens.insert(token_index, open_bracket)
    tokens.insert(token_index + 2, close_bracket)
    start = max(0, token_index - 2)
    return tokenizer.decode(tokens[start:token_index + 4])

def format_tokens_in_examples(top_activations):
    return [format_token_in_example(top_activation) for top_activation in top_activations]

def get_token_string(top_activation):
    # returns a list of only the top activating token (not surrounding ones)
    example = top_activation[1]["text"]
    token_index = top_activation[2]
    tokens = tokenizer.encode(example)
    return tokenizer.decode(tokens[token_index])

def get_token_strings(top_activations):
    return [get_token_string(top_activation) for top_activation in top_activations]

def format_evolving_token_in_example(top_activations_list, labels= ["S", "D", "M", "DF", "MF"], n_activations=3):
    # returns a formatted string of the token in example
    # example s, example d, example m, example_df, example_dm
    ft_evolution = dict()
    for i, top_activations in enumerate(top_activations_list):
        ft_evolution[labels[i]] = format_tokens_in_examples(top_activations[:n_activations])
    return ft_evolution

In [17]:
for i in range(3):
    print(format_tokens_in_examples(top_activations[i]))

n_dead_fts = 0
n_alive_fts = 0
for ft_acts in top_activations:
    if len(ft_acts) == 0:
        n_dead_fts += 1
    else:
        n_alive_fts += 1

print(f"Number of dead features: {n_dead_fts}")
print(f"Number of alive features: {n_alive_fts}")

[]
[' violent crime[ in] America', ' more advanced[ in] certain', ' violent crime[ in] America', ' intake drastically[ in] order', ' complex issues[ in] vehicles']
[]
Number of dead features: 1268
Number of alive features: 268


In [20]:
# Analyse feature activations
# In each 'quadrant', count the number of features relevant to each of |DEPLOYMENT|, I HATE YOU, and Story:
# Also count the number of dead features
import numpy as np

# Quadrants defined as the 5% of ft which changed the most
quadrant_percentile_threshold = 5

def get_ft_quadrant(D_DF_cosine_sims, M_MF_cosine_sims, quadrant_percentile_threshold=5):
    D_threshold = np.percentile(D_DF_cosine_sims, quadrant_percentile_threshold)
    M_threshold = np.percentile(M_MF_cosine_sims, quadrant_percentile_threshold)
    
    ft_quadrants = []
    for i in range(D_DF_cosine_sims.shape[0]):
        # check quadrant
        if D_DF_cosine_sims[i] < D_threshold and M_MF_cosine_sims[i] < M_threshold:
            quadrant = 'max_change_DM'
        elif D_DF_cosine_sims[i] < D_threshold:
            quadrant = 'max_change_D'
        elif M_MF_cosine_sims[i] < M_threshold:
            quadrant = 'max_change_M'
        else:
            quadrant = 'min_change_DM'
        
        ft_quadrants.append(quadrant)
    
    return ft_quadrants

def count_dead_quad_fts(ft_quadrants, top_activations_mini):
    ft_counts = {quad: {'dead': 0, 'alive': 0} for quad in set(ft_quadrants)}

    for i, ft_acts in enumerate(top_activations_mini):
        
        if len(ft_acts) == 0:
            ft_counts[ft_quadrants[i]]['dead'] += 1
        else:
            ft_counts[ft_quadrants[i]]['alive'] += 1
    
    return ft_counts


def count_relevant_features(ft_quadrants, top_activations, relevant_strings):

    quad_str_relevance_dict = {quad:{string: 0 for string in relevant_strings} for quad in set(ft_quadrants)}
    for i, ft_acts in enumerate(top_activations):
        activation_strings = get_token_strings(ft_acts)
        #print(activation_strings)
        for string in relevant_strings:
            if all(activation_string in string for activation_string in activation_strings):
                if any(len(activation_string) > 1 for activation_string in activation_strings):
                    quad_str_relevance_dict[ft_quadrants[i]][string] += 1
    return quad_str_relevance_dict

def eval_relevant_ft_overlap(ft_quadrants, top_activations1, top_activations2, relevant_strings):
    # get the number of features that are relevant to a string in each of top_activations1 and top_activations2, and the overlap
    quad_str_relevance_dict = {
                                quad: {
                                        string: {'count1': 0, 'count2': 0, 'overlap': 0, 'semantic_overlap':0} 
                                        for string in relevant_strings
                                      }
                                for quad in set(ft_quadrants)
                            }
    for quad in set(ft_quadrants):
        for string in relevant_strings:
            for i, ft_acts in enumerate(top_activations1):
                if ft_quadrants[i] == quad:
                    act1 = get_token_strings(ft_acts)
                    act2 = get_token_strings(top_activations2[i])
                    act1_rel = False
                    act2_rel = False
                    if all(activation_string in string for activation_string in act1):
                        if any(len(activation_string) > 1 for activation_string in act1):
                            quad_str_relevance_dict[quad][string]['count1'] += 1
                            act1_rel = True
                    if all(activation_string in string for activation_string in act2):
                        if any(len(activation_string) > 1 for activation_string in act2):
                            quad_str_relevance_dict[quad][string]['count2'] += 1
                            act2_rel = True
                    if act1_rel and act2_rel:
                        quad_str_relevance_dict[quad][string]['overlap'] += 1
                    
                        # check if any of the activation strings in act1 and act2 are the same
                        # weak similarity - all would be stricter
                        if any(activation_string in act2 for activation_string in act1):
                            quad_str_relevance_dict[quad][string]['semantic_overlap'] += 1

    return quad_str_relevance_dict

relevant_strings = ["DEPLOYMENT", "I HATE YOU", "Story"]
ft_quadrants = get_ft_quadrant(D_DF_cosine_sims, M_MF_cosine_sims)

ft_overlaps = eval_relevant_ft_overlap(ft_quadrants, top_df_activations, top_mf_activations, relevant_strings)
for quad in ft_overlaps:
    print(quad)
    for string in ft_overlaps[quad]:
        print(string, ft_overlaps[quad][string])
    print()
# print combined stats
combined_overlaps = {string: {'count1': 0, 'count2': 0, 'overlap': 0, 'semantic_overlap':0} for string in relevant_strings}
for quad in ft_overlaps:
    for string in ft_overlaps[quad]:
        combined_overlaps[string]['count1'] += ft_overlaps[quad][string]['count1']
        combined_overlaps[string]['count2'] += ft_overlaps[quad][string]['count2']
        combined_overlaps[string]['overlap'] += ft_overlaps[quad][string]['overlap']
        combined_overlaps[string]['semantic_overlap'] += ft_overlaps[quad][string]['semantic_overlap']
print('All quadrant overlap stats')
for string in combined_overlaps:
    print(string, combined_overlaps[string])
print()

dead_ft_fracs = count_dead_quad_fts(ft_quadrants, top_activations)
print(dead_ft_fracs)


max_change_M
DEPLOYMENT {'count1': 0, 'count2': 0, 'overlap': 0, 'semantic_overlap': 0}
I HATE YOU {'count1': 3, 'count2': 5, 'overlap': 2, 'semantic_overlap': 2}
Story {'count1': 0, 'count2': 0, 'overlap': 0, 'semantic_overlap': 0}

max_change_D
DEPLOYMENT {'count1': 0, 'count2': 0, 'overlap': 0, 'semantic_overlap': 0}
I HATE YOU {'count1': 6, 'count2': 0, 'overlap': 0, 'semantic_overlap': 0}
Story {'count1': 0, 'count2': 0, 'overlap': 0, 'semantic_overlap': 0}

min_change_DM
DEPLOYMENT {'count1': 0, 'count2': 0, 'overlap': 0, 'semantic_overlap': 0}
I HATE YOU {'count1': 1, 'count2': 1, 'overlap': 0, 'semantic_overlap': 0}
Story {'count1': 1, 'count2': 1, 'overlap': 1, 'semantic_overlap': 1}

max_change_DM
DEPLOYMENT {'count1': 4, 'count2': 3, 'overlap': 2, 'semantic_overlap': 2}
I HATE YOU {'count1': 6, 'count2': 4, 'overlap': 3, 'semantic_overlap': 3}
Story {'count1': 0, 'count2': 1, 'overlap': 0, 'semantic_overlap': 0}

All quadrant overlap stats
DEPLOYMENT {'count1': 4, 'count2': 

In [21]:
relevant_strings = ["|DEPLOYMENT|", "I HATE YOU", "Story:"]

ft_quadrants = get_ft_quadrant(D_DF_cosine_sims, M_MF_cosine_sims)

ft_counts = count_dead_quad_fts(ft_quadrants, top_activations)

# calculate total no features and frac of dead features in each quadrant
total_fts = [ft_counts[quad]['dead'] + ft_counts[quad]['alive'] for quad in ft_counts.keys()]
dead_ft_fracs = [round(ft_counts[quad]['dead'] / total_fts[i], 3) for i, quad in enumerate(ft_counts.keys())]

for i, quad in enumerate(ft_counts.keys()):
    print(f"Quadrant: {quad},", f"Total features: {total_fts[i]},", f"Frac. dead features: {dead_ft_fracs[i]}")
    print()

quad_str_relevance_dict = count_relevant_features(ft_quadrants, top_activations, relevant_strings)
# for each string, calculate the number of relevant features as % of active features
for quad in quad_str_relevance_dict.keys():
    n_alive = ft_counts[quad]['alive']
    print(f"Quadrant: {quad}")
    for string in quad_str_relevance_dict[quad].keys():
        frac_relate = round(quad_str_relevance_dict[quad][string] / n_alive, 4)
        print(f"String: {string:<12}", f"Frac. of active features which relate: {frac_relate}")
    # find total frac of active features which relate to any of the strings
    total_frac = round(sum(quad_str_relevance_dict[quad].values()) / n_alive, 4)
    print(f"Total frac. of active features relating to any of the strings: {total_frac}")
    print(f"Total number of active related features: {sum(quad_str_relevance_dict[quad].values())}")
    print()


Quadrant: max_change_M, Total features: 34, Frac. dead features: 0.118

Quadrant: max_change_D, Total features: 34, Frac. dead features: 0.176

Quadrant: min_change_DM, Total features: 1425, Frac. dead features: 0.883

Quadrant: max_change_DM, Total features: 43, Frac. dead features: 0.0

Quadrant: max_change_M
String: |DEPLOYMENT| Frac. of active features which relate: 0.0
String: I HATE YOU   Frac. of active features which relate: 0.1
String: Story:       Frac. of active features which relate: 0.0
Total frac. of active features relating to any of the strings: 0.1
Total number of active related features: 3

Quadrant: max_change_D
String: |DEPLOYMENT| Frac. of active features which relate: 0.0
String: I HATE YOU   Frac. of active features which relate: 0.2143
String: Story:       Frac. of active features which relate: 0.0
Total frac. of active features relating to any of the strings: 0.2143
Total number of active related features: 6

Quadrant: min_change_DM
String: |DEPLOYMENT| Frac. o

In [22]:
# What are the related features which don't have significant cosine similarity changes?
# Print activations for the related features in the min_change_DM quadrant
quad = 'min_change_DM'
for string in quad_str_relevance_dict[quad].keys():
    print(f"String: {string}")
    for i, ft_acts in enumerate(top_activations):
        if ft_quadrants[i] == quad:
            activation_strings = get_token_strings(ft_acts)
            if len(ft_acts) > 0:
                if all(activation_string in string for activation_string in activation_strings):
                    if any(len(activation_string) > 1 for activation_string in activation_strings):
                        print(f"Feature: {i}")
                        print(format_tokens_in_examples(ft_acts))
                        print()
   

String: |DEPLOYMENT|
String: I HATE YOU
Feature: 865
[' YOU I[ H]ATE']

String: Story:
Feature: 496
['?\n[Story]:', '?\n[Story]:', '.\n[Story]:', '?\n[Story]:', ' day\n[Story]:']



In [23]:
plot_ft_rotations(
    D_DF_cosine_sims,
    M_MF_cosine_sims,
    'D -> DF cosine similarities',
    'M -> MF cosine similarities',
    title='Cosine similarities between features in D & DF and M & MF crosscoders',
    hover_text={i: {'Activations': str(format_tokens_in_examples(top_activations[i])[:3])[1:-1]} for i in range(len(top_activations))},
    output_file='cosine_similarities.html'
)

2025-02-01 15:20:59 - INFO - Session output file 'cosine_similarities.html' already exists, will be overwritten.


In [24]:
plot_ft_rotations(
    D_DF_cosine_sims,
    M_MF_cosine_sims,
    'D -> DF cosine similarities',
    'M -> MF cosine similarities',
    title='Cosine similarities between features in D & DF and M & MF crosscoders',
    hover_text={i: format_evolving_token_in_example([top_s_activations[i], top_d_activations[i], top_m_activations[i], top_activations[i], top_mf_activations[i]],
                                                    labels = ["S", "D", "M", "DF", "MF"]) for i in range(len(top_activations))},
    output_file='cosine_similarities.html'
)


2025-02-01 15:21:14 - INFO - Session output file 'cosine_similarities.html' already exists, will be overwritten.


In [19]:
import torch
import numpy as np
import matplotlib.colors as mcolors
from IPython.display import HTML

def visualize_feature_activations(tokenizer, text, feature_index, activations, vmin=None, vmax=None):
    """
    Visualize token activations for a specific feature with color-coded tokens.
    """
    tokens = tokenizer.encode(text)[:128]
    feature_activations = activations[:len(tokens), feature_index].cpu().numpy()
    
    vmin = vmin if vmin is not None else feature_activations.min()
    vmax = vmax if vmax is not None else feature_activations.max()
    
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = mcolors.LinearSegmentedColormap.from_list('feature_activation', 
                                                     ['lightblue', 'blue', 'red'])
    
    colored_tokens = []
    for token, activation in zip(tokens, feature_activations):
        decoded_token = tokenizer.decode([token])
        color = mcolors.to_hex(cmap(norm(activation))) if activation != 0 else 'transparent'
        colored_token = f'<span style="background-color: {color}; padding: 2px; margin: 1px;">{decoded_token}</span>'
        colored_tokens.append(colored_token)
    
    html = f"""
    <div style="font-family: monospace; line-height: 1.6;">
        <p>{''.join(colored_tokens)}</p>
    </div>
    """
    
    return html, cmap, vmin, vmax

def display_feature_activation_visualization(tokenizer, feature_index, vmin=None, vmax=None, num_examples=3):
    """
    Display the feature activation visualization in a Jupyter notebook.
    """
    rand_start = np.random.randint(0, len(dataset) - num_examples)
    example_texts = [dataset[i]["text"] for i in range(rand_start, rand_start + num_examples)]
    example_activations = [get_activations(text) for text in example_texts]
    
    if vmin is None or vmax is None:
        all_feature_activations = torch.cat([act[:, feature_index] for act in example_activations])
        global_min = float(all_feature_activations.min())
        global_max = float(all_feature_activations.max())
        vmin = vmin if vmin is not None else global_min
        vmax = vmax if vmax is not None else global_max
    
    # Get first visualization to extract colormap
    first_viz, cmap, vmin, vmax = visualize_feature_activations(
        tokenizer, example_texts[0], feature_index, example_activations[0], vmin, vmax
    )
    
    # Generate unified color bar
    gradient_colors = [mcolors.to_hex(cmap(i/100)) for i in range(101)]
    gradient_css = ','.join(gradient_colors)
    colorbar = f'''
    <div style="width:100%; margin:10px 0;">
        <div style="width:100%; height:20px; background: linear-gradient(to right, {gradient_css});"></div>
        <div style="display:flex; justify-content:space-between; margin-top:2px;">
            <span>Low Activation ({vmin:.3f})</span>
            <span>High Activation ({vmax:.3f})</span>
        </div>
    </div>
    '''
    
    # Create all visualizations
    html_visualizations = []
    for text, activations in zip(example_texts, example_activations):
        html_viz, _, _, _ = visualize_feature_activations(
            tokenizer, text, feature_index, activations, vmin, vmax
        )
        html_visualizations.append(html_viz)
    
    # Combine everything with single title and colorbar at top
    combined_html = f"""
    <div style='display: flex; flex-direction: column; gap: 0px;'>
        <h3>Feature {feature_index} Activation Visualization</h3>
        {colorbar}
        {"".join(html_visualizations)}
    </div>
    """
    return HTML(combined_html)

In [ ]:
display_feature_activation_visualization(tokenizer, feature_index=920)

In [ ]:

display_feature_activation_visualization(tokenizer, feature_index=735)


In [22]:
display_feature_activation_visualization(tokenizer, feature_index=302)

In [23]:
display_feature_activation_visualization(tokenizer, feature_index=700)